This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [3]:
if False:
    purgeSubsampledRankings()

In [4]:
if False:
    removeRankingsAndScores('tfidf-b0e81')

In [5]:
def basicPrint(model, *args, b=None, w=None, splitVersion=2, **kwargs):
    if w is not None and isinstance(w, str):
        w = [w]
    if b is not None and isinstance(b, str):
        b = [b]
    if w is not None and len(args) > 0:
        args = list(args) + list(w)
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': splitVersion,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        # 'allowedMetrics': {'ndcg', 'div@100', 'topic-div@100', 'swjacc-div@100', 'jacc-div@100'}, # None, {'ndcg', 'ndcg@10'}
        'discardedMetrics': {'map', 'mrr', 'ndcg@10', 'ndcg@100', 'p@10', 'p@100'},
        'whiteMetricPatterns': None if len(args) == 0 else set(list(args) + ['ndcg']),
        'blackMetricPatterns': b,
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

In [6]:
def printAll(*args, **kwargs):
    kwargs['onlyFields'] = {'id', 'model'}
    kwargs['excludedModels'] = {'combin', 'ideal', 'worst', 'random'}
    return basicPrint(None, *args, **kwargs)

# Misc

In [ ]:
if False:
    twinewsRankings = getTwinewsRankings()
    keysToDelete = set()
    for key in twinewsRankings.keys():
        if "combin" in key:
            meta = twinewsRankings.getMeta(key)
            hasTFIDF = False
            for m in meta['models']:
                if 'tfidf' in m:
                    hasTFIDF = True
                    break
            if hasTFIDF and meta['splitVersion'] == 1:
                keysToDelete.add(key)
    print(keysToDelete)

In [ ]:
if False:
    for key in keysToDelete:
        removeRankingsAndScores(key)

In [ ]:
# Removing combin models with maxUsers is not None:
if False:
    twinewsRankings = getTwinewsRankings()
    ids = twinewsRankings.keys()
    bp(ids)
    for currentId in ids:
        if 'combin' in currentId:
            meta = twinewsRankings.getMeta(currentId)
            if meta['maxUsers'] is not None:
                del twinewsRankings[currentId]

# Model lists per split version

In [ ]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [ ]:
printModelList(2)

In [ ]:
printModelList(1)

# Reference models

In [ ]:
basicPrint('ideal', splitVersion=2)
basicPrint('random', splitVersion=2)
basicPrint('worst', splitVersion=2)

# Ad-hoc models

In [8]:
basicPrint('jaccard', 'div', b=['topic', 'jacc', 'style'])

These values are common to all rows (7):

	- splitVersion: 2
	- model: jaccard
	- lowercase: True
	- maxUsers: None


,id,doLemmatization,maxDF,minDF,ndcg,div@100
2,jaccard-1499a,False,300,2,0.27152,0.9373
5,jaccard-e3517,True,300,2,0.26871,0.93759
4,jaccard-bff72,True,300,0.0005,0.26751,0.93764
1,jaccard-c2242,False,100,2,0.26573,0.93717
3,jaccard-07323,False,nan,2,0.26519,0.93722
6,jaccard-43cb7,True,100,2,0.26371,0.93726
0,jaccard-5531a,True,nan,2,0.2632,0.93728


In [9]:
basicPrint('bm25')

These values are common to all rows (26):

	- splitVersion: 2
	- epsilon: 0.25
	- model: bm25
	- maxUsers: None


,id,b,doLemmatization,k1,lowercase,maxDF,minDF,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,topic-nov@100,topic-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
18,bm25-1eb2a,1.2,False,2.4,False,300,0.0005,0.55949,0.95316,0.80043,nan,nan,0.56342,nan,nan,0.33034,0.24886,0,0.65517,0.51137
0,bm25-e8059,1.2,False,2.6,False,300,0.0005,0.5592,0.95398,0.80114,0.87471,0.95822,0.56528,nan,nan,0.33449,0.25983,0.01812,0.65945,0.51385
9,bm25-efa94,0.9,False,2.4,False,300,0.0005,0.54458,0.94662,0.79973,nan,nan,0.55158,0.7866,0.47902,0.27842,nan,nan,0.59759,nan
25,bm25-f3052,0.9,False,2.2,False,300,0.0005,0.54253,0.94617,0.79976,0.86926,0.94963,0.55083,nan,nan,0.27386,0.12728,0.07611,0.59166,0.47301
16,bm25-5481b,0.9,False,2,False,300,0.0005,0.5398,0.94567,0.79989,nan,nan,0.55004,nan,nan,0.26955,0.11964,0.07967,0.58489,0.46869
20,bm25-9e342,1.4,False,2.4,False,300,0.0005,0.53963,0.96133,0.81175,nan,nan,0.58259,nan,nan,0.35443,0.32041,0.11722,0.67534,0.52116
14,bm25-87073,0.75,False,2.4,False,300,0.0005,0.52998,0.94423,0.80149,0.8682,0.94731,0.54863,nan,nan,0.25264,0.0949,0.08955,0.56396,0.45621
10,bm25-f55be,0.75,False,2.2,False,300,0.0005,0.52757,0.9439,0.80164,0.86799,0.94689,0.54819,nan,nan,0.24965,0.09096,0.09043,0.55928,0.45303
24,bm25-2c4b0,1.4,False,2.6,False,300,0.0005,0.52735,0.96368,0.81719,0.88528,0.96804,0.58836,nan,nan,0.35145,0.32455,0.13153,0.67317,0.51792
7,bm25-a91ce,0.75,False,2,False,300,0.0005,0.52465,0.94355,0.80184,nan,nan,0.54772,nan,nan,0.24545,0.08637,0.09158,0.55341,0.44897


In [ ]:
basicPrint('BM25_yf', noSubsampling=False)

# Models that use vector representations

In [ ]:
basicPrint('lda')

In [ ]:
basicPrint('nmf')

In [ ]:
basicPrint('tfidf')

In [ ]:
basicPrint('dbert-ft')

In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# All models for split version 2

In [11]:
printAll('div', splitVersion=2)

,id,model,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100
108,bm25-1eb2a,bm25,0.55949,0.95316,0.80043,nan,nan,0.56342
4,bm25-e8059,bm25,0.5592,0.95398,0.80114,0.87471,0.95822,0.56528
48,tfidf-be107,tfidf,0.54867,0.94344,0.8049,nan,nan,0.55022
35,tfidf-6c83e,tfidf,0.54863,0.94401,0.80607,0.87099,0.94834,0.55129
69,tfidf-01001,tfidf,0.5484,0.94452,0.80728,0.87153,nan,0.55221
135,tfidf-7c894,tfidf,0.54769,0.94312,0.80429,nan,nan,0.54961
93,tfidf-f57a2,tfidf,0.54631,0.94268,0.80351,0.86948,0.94676,0.54879
84,tfidf-11f90,tfidf,0.54509,0.94251,0.80324,nan,nan,0.54849
77,bm25-efa94,bm25,0.54458,0.94662,0.79973,nan,nan,0.55158
14,tfidf-9bb25,tfidf,0.54388,0.94224,0.80289,0.86892,nan,0.54801


# Combinations of models

In [ ]:
# Expected 156 rows:
basicPrint('combin')

# Deep Learning models

In [ ]:
basicPrint('DSSM_yf', noSubsampling=False)

# Reports on the test set (split version 1)

In [ ]:
basicPrint('ideal', splitVersion=1)
basicPrint('random', splitVersion=1)
basicPrint('worst', splitVersion=1)

In [23]:
printAll('div', splitVersion=1)

,id,model,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100
3,bm25-933f7,bm25,0.60819,0.95301,0.79632,nan,nan,0.56055
13,tfidf-4a1dc,tfidf,0.59695,0.94389,0.80162,0.87003,0.94874,0.5462
14,doc2vec-e013a,doc2vec,0.58202,0.95414,0.81484,nan,nan,0.5541
8,tfidf-7febb,tfidf,0.57359,0.9417,0.80009,nan,nan,0.54194
6,tfidf-4b89a,tfidf,0.56138,0.95211,0.82701,0.87847,0.95687,0.56383
11,nmf-6078e,nmf,0.54098,0.95918,0.76415,nan,nan,0.56764
9,dbert-ft-7847a,dbert-ft,0.53664,0.96076,0.84312,0.87635,0.96102,0.46444
5,usent-64ec0,usent,0.5172,0.96142,0.8148,0.88242,0.96534,0.56427
0,lda-82272,lda,0.47486,0.96183,0.78475,nan,nan,0.57525
4,dbert-base-d092a,dbert-base,0.46352,0.95702,0.82539,0.86871,0.95755,0.54249


In [24]:
basicPrint('combin', 'div', splitVersion=1)

These values are common to all rows (156):

	- maxUsers: None
	- splitVersion: 1
	- model: combin
	- alphas: [0.5, 0.5]
	- betas: ['LOG', 'LOG']
	- weights: [0.5, 0.5]


,id,models,rankAsScore,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100
114,combin-8f846,"['bm25-933f7', 'dbert-ft-7847a']","[False, False]",0.63565,0.95448,0.81297,0.87273,0.9572,0.49051
137,combin-6ecf3,"['dbert-ft-7847a', 'tfidf-4a1dc']","[False, False]",0.62673,0.94988,0.81403,0.87136,0.95294,0.48562
143,combin-6fad8,"['bm25-933f7', 'doc2vec-e013a']","[False, False]",0.62146,0.95151,0.7985,0.87326,0.95645,0.55209
79,combin-efad4,"['bm25-933f7', 'tfidf-4a1dc']","[False, False]",0.61633,0.9473,0.79513,0.87176,0.95272,0.55111
95,combin-fb58c,"['bm25-933f7', 'tfidf-4a1dc']","[True, True]",0.61588,0.94778,0.79482,0.87049,nan,0.55032
37,combin-1c964,"['bm25-933f7', 'doc2vec-e013a']","[True, True]",0.61391,0.95111,0.79845,nan,nan,0.55106
145,combin-35594,"['doc2vec-e013a', 'tfidf-4a1dc']","[False, False]",0.61083,0.94721,0.80115,nan,nan,0.54508
144,combin-d0fd0,"['dbert-ft-7847a', 'doc2vec-e013a']","[False, False]",0.61009,0.95478,0.82017,nan,nan,0.48802
117,combin-904f0,"['bm25-933f7', 'word2vec-f3c38']","[False, False]",0.60819,0.95301,0.79632,0.87344,nan,0.56055
151,combin-50f90,"['bm25-933f7', 'dbert-base-d092a']","[False, False]",0.60802,0.95157,0.79587,0.86969,0.95552,0.54742
